In [22]:
import random
import numpy as np

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))


# производная сигмодидальной функции
def sigmoid_pr(x):
    return sigmoid(x) * (1-sigmoid(x))

class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        
        # Задаём начальные смещения
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        
        # Указаны начальные веса: в sizes указывается 
        # 1) количество элементов входного слоя, 2) количество элементов внутреннего слоя, 
        # 3) кол-во элеметов выходного слоя        
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
    
    # Возвращает результат работы нейросети
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    # Обучение нейросети
    def SGD(self, train, epochs, minibatch_size, eta, test):
        test =  list(test)
        len_test = len(test)
        train = list(train)
        len_train = len(train)
        
        for i in range(epochs):
            random.shuffle(train)
            # создание листа из подвыборок из train
            mini_batches = [train[k:k+minibatch_size] for k in range(0, len_train, minibatch_size)]
            for mini_batch in mini_batches:
                # один шаг градиентного спуска
                self.update_mini_batch(mini_batch, eta)
            print(i, " ", self.evaluate(test), len_test)
                
                
    def update_mini_batch(self, mini_batch, eta):
        # список градиентов для каждого слоя
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            d_nablab, d_nablaw = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, d_nablab)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, d_nablaw)]
            self.weights = [w - (eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b - (eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
            
    
    def backprop(self, X, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = X
        activations = [X]
        zs = []
        
        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_pr(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for i in range(2, self.num_layers):
            z = zs[-i]
            sp = sigmoid_pr(z)
            delta = np.dot(self.weights[-i+1].transpose(), delta) * sp
            nabla_b[-i] = delta
            nabla_w[-i] = np.dot(delta, activations[-i-1].transpose())
        return nabla_b, nabla_w
    
    # количество правильных результатов работы нейросети  
    def evaluate(self, test):
        # test_result[0] - вектор с изображением цифры
        # test_result[1] - значение цифры на картинке
        # Ответ нейросети - номер нейрона в выходном слое, имеющего 
        # наибольшее значение функции активации
        test_results = [(np.argmax(self.feedforward(X)), y) for (X, y) in test]
        return sum(int(X==y) for (X, y) in test_results)
    
    # считает вектор частных производных в градиенте
    def cost_derivative(self, output_activations, y):
        return output_activations - y
                
    
        
        
net = Network([2, 3, 1])
print('Число слоев ', net.num_layers)
for i in range(net.num_layers):
    print('Число нейронов в слое ', i, ': ', net.sizes[i])
for i in range(net.num_layers-1):
    print('w ', i+1, ':', np.round(net.weights[i], 2))
    print('b ', i+1, ':', np.round(net.weights[i], 2))


Число слоев  3
Число нейронов в слое  0 :  2
Число нейронов в слое  1 :  3
Число нейронов в слое  2 :  1
w  1 : [[ 0.95  0.16]
 [ 0.17  0.01]
 [-1.2  -0.28]]
b  1 : [[ 0.95  0.16]
 [ 0.17  0.01]
 [-1.2  -0.28]]
w  2 : [[-0.16  0.02 -2.51]]
b  2 : [[-0.16  0.02 -2.51]]


In [23]:
import gzip
import pickle


def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    train_data, valid_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (train_data, valid_data, test_data)

# преобразование train в список из 50000 пар (X, y) (Признаки - метки)
def load_data_wrapper():
    train, valid, test = load_data()
    train_input = [np.reshape(X, (784, 1)) for X in train[0]]
    train_result = [vectorized_result(y) for y in train[1]]
    train_data = zip(train_input, train_result)
    
    valid_input = [np.reshape(X, (784, 1)) for X in valid[0]]
    valid_data = zip(valid_input, valid[1])
    
    test_input = [np.reshape(X, (784, 1)) for X in test[0]]
    test_data = zip(test_input, test[1])
    return train_data, valid_data, test_data


def vectorized_result(i):
    e = np.zeros((10, 1))
    e[i] = 1.0
    return e

In [24]:
train, valid, test = load_data_wrapper()
net = Network([784, 30, 10])
net.SGD(train, 30, 10, 3.0, test=test)

0   7894 10000
1   7968 10000
2   8564 10000
3   8839 10000
4   8982 10000
5   8964 10000
6   9004 10000
7   9028 10000
8   9094 10000
9   9014 10000
10   9083 10000
11   9020 10000
12   9130 10000
13   9208 10000
14   9156 10000
15   9148 10000
16   9199 10000
17   9163 10000
18   9194 10000
19   9162 10000


<ipython-input-22-95f6603b3faf>:5: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-x))


20   9117 10000
21   9187 10000
22   9056 10000
23   9197 10000
24   9101 10000
25   9176 10000
26   9195 10000
27   9134 10000
28   9228 10000
29   9263 10000
